**IMPORT LIBRARY**

In [22]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
root = "C:\\Users\ismmk\data\grantibbva"
USER_ID = 'user_id' 
TARGET = 'moved_after_2019'

**IMPORT DATASETS**

In [23]:
df_skills = pd.read_csv("data/skills.csv")
df_education = pd.read_csv('data/education.csv')
df_languages = pd.read_csv('data/languages.csv')
df_work_experiences = pd.read_csv('data/work_experiences.csv')

###  Language

In [3]:
df_work_experiences = df_work_experiences[df_work_experiences['start_year_month'] < 201901] 

In [4]:
df_languages = df_languages[df_languages['language'].notnull() & df_languages['proficiency'].notnull()]

In [5]:
#df_languages.loc = df_languages.applymap(lambda x: encode_proficiency(x))

def encode_proficiency(value):
    if value == 'elementary':
        return 1
    elif value == 'limited_working':
        return 2
    elif value == 'professional_working':
        return 3
    elif value == 'full_professional':
        return 4
    elif value == 'native_or_bilingual':
        return 5
    else:
        return 0
    
df_languages['proficiency'] = df_languages['proficiency'].apply(encode_proficiency)

In [6]:
df_languages

,user_id,language,proficiency
0,8,İngilizce,4
1,8,Türkçe,5
2,8,Fransızca,1
4,11,Turkish,5
5,11,English,3
...,...,...,...
76054,66262,English,4
76058,66272,English,3
76059,66273,Türkçe,5
76060,66273,İngilizce,3


In [7]:
df_languages.language.value_counts().head(60)

İngilizce                              17029
English                                12253
Türkçe                                 11925
Turkish                                 7625
Almanca                                 3492
German                                  2888
French                                   855
İspanyolca                               813
Spanish                                  796
Fransızca                                705
Rusça                                    498
Russian                                  485
Arabic                                   468
ingilizce                                428
Arapça                                   333
Italian                                  322
İtalyanca                                308
Japanese                                 257
İngilizce, Orta (1100-1500)              196
Deutsch                                  196
Japonca                                  150
İngilizce                                144
Azerbaijan

In [8]:
df_languages.loc[df_languages['language'].str.contains("Eng|eng|İng|ing|Ing|English|english|İngilizce|ingilizce|Ingilizce|İNGİLİZCE|INGILIZCE"), 'language'] = "English"
df_languages.loc[df_languages['language'].str.contains("Tür|Tur|tür|tur|TURKISH|TÜRKÇE|Tükçe|tr"), 'language'] = "Turkish"
df_languages.loc[df_languages['language'].str.contains("Alm|alm|ger|Ger|Deu|deu|ALMANCA|GER|DEU|ALM"), 'language'] = "German"
df_languages.loc[df_languages['language'].str.contains("Fr|fr|FR"), 'language'] = "French"
df_languages.loc[df_languages['language'].str.contains("Spa|spa|İsp|isp|SPA|ISP|İSP|Español"), 'language'] = "Spanish"
df_languages.loc[df_languages['language'].str.contains("Rus|rus|RUS"), 'language'] = "Russian"
df_languages.loc[df_languages['language'].str.contains("ara|Ara|ARA"), 'language'] = "Arabic" 
df_languages.loc[df_languages['language'].str.contains("Chinese|CHINESE|CHI|Chi|chi|Çince"), 'language'] = "Chinese"
df_languages.loc[df_languages['language'].str.contains("Japanese|Japonca|japa|japo|JAPA|JAPO"), 'language'] = "Japanese"
df_languages.loc[df_languages['language'].str.contains("İtalyanca|Italian|itali|ital|ITA|İTAL"), 'language'] = "Italian"
df_languages.loc[df_languages['language'].str.contains("Azerice|Azerbaijani|Azeri|Azərba"), 'language'] = "Azerbaijani"
df_languages.loc[df_languages['language'].str.contains("Kurdish|Kürt|kürtçe|kurdish"), 'language'] = "kurdish"
df_languages.loc[df_languages['language'].str.contains("Portekizce|Portuguese|por|POR"), 'language'] = "Portuguese"
df_languages.loc[df_languages['language'].str.contains("korece|Korean|kor|KOR|KORECE|KOREAN|Korece"), 'language'] = "Korean"

df_languages.loc[~df_languages["language"].isin(["English","Turkish","German","French","Spanish","Russian","Arabic","Korean","Chinese","Japanese","Italian","Azerbaijani","kurdish","Portuguese"]), "language"] = "OtherLanguage"

In [9]:
df_languages.language.value_counts().head(50)

English          30442
Turkish          19864
German            6686
Spanish           1674
French            1610
OtherLanguage     1525
Russian           1001
Arabic             829
Italian            654
Japanese           409
Azerbaijani        204
Chinese            176
Korean             141
kurdish            114
Portuguese          72
Name: language, dtype: int64

In [10]:
df_languages = df_languages.drop_duplicates(['user_id', 'language'])
df_languages = pd.pivot(df_languages, index='user_id', columns='language', values='proficiency')
df_languages = df_languages.fillna(0).astype(int)
df_languages.head()

language,Arabic,Azerbaijani,Chinese,English,French,German,Italian,Japanese,Korean,OtherLanguage,Portuguese,Russian,Spanish,Turkish,kurdish
user_id,,,,,,,,,,,,,,,
8,0,0,0,4,1,0,0,0,0,0,0,0,0,5,0
11,0,0,0,3,0,0,0,0,0,0,0,0,0,5,5
12,0,0,0,3,0,0,0,0,0,0,0,0,0,5,0
13,0,0,0,4,0,2,0,0,0,0,0,0,0,0,0
14,0,0,0,3,0,0,0,0,0,0,0,0,0,5,0


###  Education

In [11]:
df_education = df_education[df_education['school_name'].notnull() & df_education['degree'].notnull()]

In [12]:
df_education.loc[df_education['degree'].str.contains("Associate|Ön|ön lisans|önlisans", na=False), 'degree'] = "ÖNLİSANS"
df_education.loc[df_education['degree'].str.contains("Doctor|Doktor|Ph", na=False), 'degree'] = "DOKTORA"
df_education.loc[df_education['degree'].str.contains("Yüksek|Master|MSc|MS|M.Sc.|MBA|Msc|M.Sc|M.S.|M.S", na=False), 'degree'] = "YÜKSEK_LİSANS"
df_education.loc[df_education['degree'].str.contains("BS|Bs|Bachelor|BSc|BE|B.E.|B.Sc.|B.S.|B.S|Engineer|BA|BBA|BEng|B.B.A.|B.A.Sc.|Undergraduate|Licentiate|Licence|License|Lisans|lisans|Bacheleer|bachelor|Bsc", na=False), 'degree'] = "LİSANS"

df_education.loc[~df_education["degree"].isin(["ÖNLİSANS","DOKTORA","YÜKSEK_LİSANS","LİSANS"]), "degree"] = "Diğer"

In [13]:
df_education = df_education.drop_duplicates(['user_id', 'degree'])
df_education = pd.pivot(df_education, index='user_id', columns='degree', values='school_name')
df_education.head()

degree,DOKTORA,Diğer,LİSANS,YÜKSEK_LİSANS,ÖNLİSANS
user_id,,,,,
1,NaN,NaN,Fırat Üniversitesi,Fırat Üniversitesi,NaN
2,NaN,Fırat Üniversitesi,NaN,NaN,NaN
3,NaN,NaN,Fırat Üniversitesi,NaN,NaN
4,NaN,NaN,Fırat Üniversitesi,NaN,NaN
5,NaN,İzmit Anadolu İmam Hatip Lisesi,The State University of Applied Sciences in El...,Firat University,NaN


### Work Experience

In [14]:
df_new = pd.DataFrame()

In [15]:
df_work_experiences = df_work_experiences.sort_values(by=['user_id', 'start_year_month'])

In [16]:
df_new['company(1th)'] = df_work_experiences.groupby(USER_ID)['company_id'].nth(-1).astype(str)
df_new['company(2th)'] = df_work_experiences.groupby(USER_ID)['company_id'].nth(-2).astype(str)
df_new['company(3th)'] = df_work_experiences.groupby(USER_ID)['company_id'].nth(-3).astype(str)
df_new['company(4th)'] = df_work_experiences.groupby(USER_ID)['company_id'].nth(-4).astype(str)


df_new['company_location(1th)'] = df_work_experiences.groupby(USER_ID)['location'].nth(-1).astype(str)
df_new['company_location(2th)'] = df_work_experiences.groupby(USER_ID)['location'].nth(-2).astype(str)
df_new['company_location(3th)'] = df_work_experiences.groupby(USER_ID)['location'].nth(-3).astype(str)
df_new['company_location(4th)'] = df_work_experiences.groupby(USER_ID)['location'].nth(-4).astype(str)


df_new['min_exp_time'] = df_work_experiences.groupby(USER_ID)['start_year_month'].min()
df_new['max_exp_time'] = df_work_experiences.groupby(USER_ID)['start_year_month'].max()


df_new['company_count_2018'] = df_work_experiences[df_work_experiences['start_year_month'].gt(201712)].groupby(USER_ID).size()
df_new['company_count_2017'] = df_work_experiences[df_work_experiences['start_year_month'].gt(201612)].groupby(USER_ID).size()
df_new['company_count_2016'] = df_work_experiences[df_work_experiences['start_year_month'].gt(201512)].groupby(USER_ID).size()

In [17]:
df_work_experiences = df_new
df_work_experiences.head()

,company(1th),company(2th),company(3th),company(4th),company_location(1th),company_location(2th),company_location(3th),company_location(4th),min_exp_time,max_exp_time,company_count_2018,company_count_2017,company_count_2016
user_id,,,,,,,,,,,,,
0,0,0,NaN,NaN,Visual Studio Asp.Net Developer,Serbest Çalışmalar,NaN,NaN,200509,200509,NaN,NaN,NaN
2,10,7,9,NaN,"Mersin, Turkey","Elazig, Turkey","Elazig, Turkey",NaN,201612,201806,1.0,2.0,3.0
5,15,NaN,NaN,NaN,İstanbul,NaN,NaN,NaN,201706,201706,NaN,1.0,1.0
7,20,21,21,21,Elazığ,Macedonia,"Istanbul, Turkey","Istanbul, Turkey",201607,201812,1.0,3.0,4.0
10,26,26,27,NaN,istanbul,"Istanbul, Turkey","Elazig, Turkey",NaN,201308,201805,2.0,2.0,2.0


###  Skill 

In [18]:
df_skill = df_skills

used_skills = df_skill['skill'].value_counts().iloc[:60].index # we only took most common 60 skills, you can increase it

df_skill = df_skill[df_skill['skill'].isin(used_skills)]
df_skill['experience'] = True

counts = df_skill.skill.value_counts()
counts.to_csv('name.csv')

df_skill = df_skill.drop_duplicates(['user_id', 'skill'])
df_skill = pd.pivot(df_skill, index='user_id', columns='skill', values='experience')
df_skill = df_skill.fillna(0).astype(int)
df_skill = df_skill.drop(["İngilizce"], axis=1)
df_skill.head()

C:\Users\ismmk\AppData\Local\Temp\ipykernel_33724\2273662330.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_skill['experience'] = True


skill,.NET,AJAX,ASP.NET,ASP.NET MVC,Agile Methodologies,Android,AutoCAD,C,C#,C++,...,Software Engineering,Spring Framework,T-SQL,Visual Studio,Web Development,Web Geliştirme,Web Services,XML,Yazılım Geliştirme,jQuery
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:

idx = 'user_id'
df_train = pd.read_csv("data/train_users.csv")
df_test  = pd.read_csv("data/test_users.csv")
df_subm  = pd.read_csv(os.path.join(root, 'submission.csv'))

df_train = df_train.set_index(USER_ID)
df_test = df_test.set_index(USER_ID)
df_subm = df_subm.set_index(USER_ID)
df_lang = df_languages
df_edu = df_education
df_exp = df_work_experiences


### MERGE

In [27]:
df_train[df_edu.columns]   = df_edu[df_edu.columns]
df_train[df_lang.columns]  = df_lang[df_lang.columns]
df_train[df_skill.columns] = df_skill[df_skill.columns]
df_train[df_exp.columns]   = df_exp[df_exp.columns]

df_test[df_edu.columns]   = df_edu[df_edu.columns]
df_test[df_lang.columns]  = df_lang[df_lang.columns]
df_test[df_skill.columns] = df_skill[df_skill.columns]
df_test[df_exp.columns]   = df_exp[df_exp.columns]

In [28]:
cat_cols = [col for col in df_test.columns if df_test[col].dtype == 'object']
num_cols = [col for col in df_test.columns if df_test[col].dtype != 'object']

In [29]:
for col in cat_cols:
    train_cats  = set(df_train[col].unique())
    test_cats   = set(df_test[col].unique())
    common_cats = set.intersection(train_cats, test_cats)
    
    df_train.loc[~df_train[col].isin(common_cats), col] = 'other'
    df_test.loc[~df_test[col].isin(common_cats), col] = 'other'

In [30]:
df_all = pd.concat([df_train, df_test], axis=0)

for col in cat_cols:
    df_all[col] = df_all[col].factorize()[0]

df_all[cat_cols] = df_all[cat_cols].astype('category')
df_all[num_cols] = df_all[num_cols].fillna(0)

df_train = df_all.loc[df_train.index, df_train.columns]
df_test = df_all.loc[df_test.index, df_test.columns]

### MODEL

In [31]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
X, y = df_train.drop(columns=[TARGET]), df_train[TARGET]
clf = RandomForestClassifier(n_estimators=100, random_state=0)
cv  = StratifiedKFold(shuffle=True, random_state=0)

scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')

print(f'Scores mean: {np.mean(scores):.4f}')
print(f'Scores std: {np.std(scores):.4f}')

Scores mean: 0.6678
Scores std: 0.0032


In [32]:
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold,StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve,classification_report,confusion_matrix
from sklearn.preprocessing import scale,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier  
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,stratify=y)

In [33]:
logistic_model = LogisticRegression(solver="lbfgs",random_state=42).fit(X_train,y_train)


In [34]:
cross_val_score(logistic_model,X_test,y_test,scoring="accuracy",cv=5)

C:\Users\ismmk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0.61259902, 0.61297624, 0.61297624, 0.61297624, 0.61297624])

### DECİSİON TREE

In [35]:
decision_tree_model = DecisionTreeClassifier(random_state=42)
params = {"max_depth":[2,3,5,8,10],"min_samples_split":[2,3,5,10,20,50]}
cv_model = GridSearchCV(decision_tree_model,cv=cv,param_grid=params,n_jobs=-1).fit(X_train,y_train)
print(cv_model.best_params_)

{'max_depth': 10, 'min_samples_split': 10}


In [36]:
final_decisionTree_model = DecisionTreeClassifier(max_depth=10,min_samples_split=2,random_state=42).fit(X_train,y_train)

In [37]:
cross_val_score(final_decisionTree_model,X_test,y_test,scoring="accuracy",cv=5)

array([0.59826481, 0.60920407, 0.60316862, 0.60241418, 0.60995851])

### RANDOM FOREST

In [38]:
randomForest_model = RandomForestClassifier(random_state=42,class_weight='balanced_subsample')
params = {"n_estimators":[200,300],
             "max_features":[3,5,7],
             "min_samples_split":[2,5,10]}

randomForest_cv_model = GridSearchCV(randomForest_model,cv=cv,param_grid=params,n_jobs=-1).fit(X_train,y_train)
print(randomForest_cv_model.best_params_)

{'max_features': 3, 'min_samples_split': 2, 'n_estimators': 300}


In [39]:
final_randomForest_model = RandomForestClassifier(n_estimators=300,class_weight='balanced_subsample',max_features=5,min_samples_split=5,random_state=42).fit(X_train,y_train)

In [40]:
cross_val_score(final_randomForest_model,X_test,y_test,scoring="accuracy",cv=5)

array([0.64051301, 0.63372312, 0.64315353, 0.64126745, 0.6378725 ])

### GBM

In [41]:
gbm_model = GradientBoostingClassifier(random_state=42)
params = {"learning_rate":[0.1,0.01,0.05],
           "n_estimators":[100,300],
           "max_depth":[2,3]}
gbm_cv_model = GridSearchCV(gbm_model,param_grid=params,cv=cv,n_jobs=-1).fit(X_train,y_train)
print(gbm_cv_model.best_params_)

{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300}


In [42]:
final_gbm_model = GradientBoostingClassifier(n_estimators=300,learning_rate=0.1,max_depth=3,random_state=42).fit(X_train,y_train)

In [43]:
cross_val_score(final_gbm_model,X_test,y_test,scoring="accuracy",cv=5)

array([0.62504715, 0.62165221, 0.62693323, 0.63145983, 0.62655602])

### XGB

In [44]:
xgb_model = XGBClassifier(tree_method="gpu_hist", enable_categorical=True)
params = {"n_estimators":[100,500],
            "max_depth":[3,5],
            "learning_rate":[0.1,0.01],
            "subsample":[0.6,0.8]
         }
xgb_cv_model = GridSearchCV(xgb_model,param_grid=params,cv=3,n_jobs=-1).fit(X_train,y_train)
print(xgb_cv_model.best_params_)

{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}


In [45]:
final_xgb_model = XGBClassifier(n_estimators=500,learning_rate=0.1,max_depth=5,subsample=0.8,tree_method="gpu_hist", enable_categorical=True).fit(X_train,y_train)

In [46]:
cross_val_score(final_xgb_model,X_test,y_test,scoring="accuracy",cv=5)

array([0.58807997, 0.59751037, 0.60128253, 0.60203697, 0.59411543])

### FİNAL PREDİCT

In [47]:
models=[final_decisionTree_model,
        final_randomForest_model,
        final_gbm_model,
        final_xgb_model]
for model in models:
    predicted=model.predict(X_test)
    score = accuracy_score(y_test,predicted)
    print("Score is: %{:.1f}".format(score*100)) 

Score is: %62.5
Score is: %67.9
Score is: %63.8
Score is: %61.0


In [48]:
models=[final_decisionTree_model,
        final_randomForest_model,
        final_gbm_model,
        final_xgb_model]
for model in models:
    predicted = final_randomForest_model.predict(df_test)
    df_subm.loc[:,"moved_after_2019"] = predicted
    print(df_subm["moved_after_2019"].value_counts() )

0.0    10574
1.0     2681
Name: moved_after_2019, dtype: int64
0.0    10574
1.0     2681
Name: moved_after_2019, dtype: int64
0.0    10574
1.0     2681
Name: moved_after_2019, dtype: int64
0.0    10574
1.0     2681
Name: moved_after_2019, dtype: int64


In [49]:
predicted=final_randomForest_model.predict(X_test)

In [50]:
score = accuracy_score(y_test,predicted)

In [51]:
print("Score is: %{:.1f}".format(score*100))  

Score is: %67.9


In [52]:
predicted = final_randomForest_model.predict(df_test)

### SUBMİSSİON

In [53]:
df_subm.loc[:,"moved_after_2019"] = predicted

In [54]:
submission = df_subm.astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
submission.to_csv("submission777.csv")

In [ ]:
randomForest_model = RandomForestClassifier(random_state=42,class_weight='balanced_subsample')
params = {"n_estimators":[200,300,500,1000],
             "max_features":[3,5,7,9,10],
             "min_samples_split":[2,5,10,20,30]}

randomForest_cv_model = GridSearchCV(randomForest_model,cv=5,param_grid=params,n_jobs=-1).fit(X_train,y_train)
print(randomForest_cv_model.best_params_)

In [ ]:
final_randomForest_model = RandomForestClassifier(n_estimators=500,class_weight='balanced_subsample',max_features=7,min_samples_split=2,random_state=42).fit(X_train,y_train)

In [ ]:
predicted=final_randomForest_model.predict(X_test)
score = accuracy_score(y_test,predicted)
print("Score is: %{:.1f}".format(score*100))  

In [ ]:
predicted = final_randomForest_model.predict(df_test)

In [ ]:
df_subm.loc[:,"moved_after_2019"] = predicted
submission = df_subm.astype(int)
submission.to_csv("submission778.csv")